<a href="https://colab.research.google.com/github/Devendarreddybathini/Devendarreddy_INFO5731_spring2023/blob/main/In_class_exercise_04_03282023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [2]:
# Write your code here
import requests
from bs4 import BeautifulSoup
import pandas as pd

header = [] # List to store Review headings
body = [] # List to store reviews

for number in range(300):
    link = "https://www.flipkart.com/redmi-10-midnight-black-64-gb/product-reviews/itmd93641e4ebb47?pid=MOBGC9GYEBH3GZ4E&lid=LSTMOBGC9GYEBH3GZ4ESWAKTT&marketplace=FLIPKART&page=" + str(number) # Generating link dynamically
    page = requests.get(link) # Accessing the webpage
    soup = BeautifulSoup(page.text, 'html.parser')
    main_reviews = soup.find_all(class_='_2-N8zT') # Getting the Review Heading by using the class name
    text_reviews = soup.find_all(class_='t-ZTKy') # Getting the full reviews by using the class name

    for ele, sub_ele in zip(main_reviews, text_reviews): # Iterating through the list
        header.append(ele.text) # Appending to header list
        body.append(sub_ele.text) # Appending to body list

df = pd.DataFrame(list(zip(header, body)), columns =['SummaryView', 'Review'])  # Creating Dataframe
print("Length of data frame is {0}".format(len(df)))
df

Length of data frame is 2980


,SummaryView,Review
0,Fabulous!,"Good phone in this price range, battery backup..."
1,Wonderful,Very nice mobile 🥳😊Performance is very good 😊C...
2,Simply awesome,Good phone very nice working and good performa...
3,Wonderful,Mobile is best in this price rangeProcessor Qu...
4,Good quality product,This phone camera is not good but you can use ...
...,...,...
2975,Awesome,Very goodREAD MORE
2976,Worth the money,"Good,READ MORE"
2977,Simply awesome,Verry nice 👍🙂READ MORE
2978,Worth every penny,Product is good in this priceREAD MORE


In [3]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 32.9 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
import re

def cleantext(Review):
    text = re.sub(r'@[A-Za-z0-9_]+', '', Review) #removes @mentions
    text = re.sub(r'#[A-Za-z0-9_]+', '', text) #removing Hashtag symbols.
    text = re.sub(r'RT[\s]+', '', text) #removing Retweet symbol (RT).
    text = re.sub(r'https?:\/\/\S+', '', text) #remove URL's.
    text = re.sub(r"www.\S+", "", text)
    text = re.sub('[()!?]', ' ', text)
    text = re.sub('\[.*?\]',' ', text)
    text = re.sub("[^A-Za-z0-9]"," ", text)#Filtering non-alphanumeric characters
    text = re.sub('\\n', '', text)#removing the '\n' character
    text = re.sub(r'[0-9]+', '', text)#removing numbers
    
    return text

df["Review"] = df["Review"].apply(cleantext)

# Convert to list
data = df.Review.values.tolist()

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [10]:
b = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
t = gensim.models.Phrases(b[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bmod = gensim.models.phrases.Phraser(b)
tmod = gensim.models.phrases.Phraser(t)

# See trigram example
print(tmod[bmod[data_words[0]]])

['good', 'phone', 'in', 'this', 'price', 'range', 'battery', 'backup', 'good', 'camera', 'ok', 'ok', 'and', 'performance', 'very', 'very', 'good', 'am', 'mi', 'love', 'and', 'very', 'fast', 'delivery', 'flipkart', 'thanksread', 'more']


In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bmod[doc] for doc in texts]

def make_trigrams(texts):
    return [tmod[bmod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['good', 'phone', 'price', 'range', 'battery', 'backup', 'good', 'camera', 'ok', 'performance', 'good', 'love', 'fast', 'delivery', 'flipkart', 'thanksread']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


In [14]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('backup', 1),
  ('battery', 1),
  ('camera', 1),
  ('delivery', 1),
  ('fast', 1),
  ('flipkart', 1),
  ('good', 3),
  ('love', 1),
  ('ok', 1),
  ('performance', 1),
  ('phone', 1),
  ('price', 1),
  ('range', 1),
  ('thanksread', 1)]]

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.239*"good" + 0.148*"read" + 0.113*"nice" + 0.087*"phone" + 0.063*"camera" + 0.043*"battery" + 0.029*"quality" + 0.025*"price" + 0.021*"superread" + 0.017*"performance"'), (1, '0.144*"bad" + 0.069*"mobileread" + 0.062*"buy" + 0.054*"super" + 0.046*"charge" + 0.030*"take" + 0.027*"month" + 0.019*"use" + 0.019*"hour" + 0.017*"work"'), (2, '0.099*"thank" + 0.070*"badread" + 0.064*"flipkartread" + 0.059*"flipkart" + 0.050*"excellent" + 0.048*"slow" + 0.044*"rangeread" + 0.044*"design" + 0.032*"normal" + 0.020*"pro"'), (3, '0.070*"performanceread" + 0.045*"sound" + 0.033*"usage" + 0.030*"bit" + 0.026*"heavy" + 0.025*"level" + 0.023*"screenread" + 0.020*"family" + 0.016*"watch" + 0.013*"basic"'), (4, '0.235*"product" + 0.177*"mobile" + 0.164*"phoneread" + 0.038*"overall" + 0.020*"thing" + 0.020*"love" + 0.019*"really" + 0.016*"ok" + 0.014*"superbread" + 0.013*"fine"'), (5, '0.427*"productread" + 0.053*"budgetread" + 0.041*"qualityread" + 0.022*"friendly" + 0.016*"part" + 0.014*"amazin

In [17]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.463703454650702

Coherence Score:  0.5376872211067603


In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.452259  0.062816       1        1  39.738700
6     -0.018064 -0.102110       2        1  10.383094
9     -0.063945  0.120651       3        1   9.590195
4     -0.012598 -0.099811       4        1   9.521130
1     -0.077301  0.152633       5        1   9.226944
7     -0.000245 -0.297878       6        1   6.498956
2     -0.056125  0.024013       7        1   6.229825
8     -0.081202  0.075847       8        1   4.633063
5     -0.064912  0.031435       9        1   2.340145
3     -0.077865  0.032403      10        1   1.837949, topic_info=             Term         Freq        Total Category  logprob  loglift
402      goodread   511.000000   511.000000  Default  30.0000  30.0000
6            good  1107.000000  1107.000000  Default  29.0000  29.0000
403      niceread   302.000000   302.000000  Default  28.0000  28.0000
79        product   261.000000   261.000000  Default  27.0000  27.0000
19           read   686.000000   686.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
51           drop     0.450028     1.471114  Topic10  -6.1633   2.8121
304         gamer     0.449492     1.470679  Topic10  -6.1645   2.8112
725      multiple     0.295106     1.316360  Topic10  -6.5852   2.5012
1467  androidread     0.375451     1.714655  Topic10  -6.3444   2.4777
1468       member     0.375451     1.714655  Topic10  -6.3444   2.4777

[340 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
595       5  0.863910        able
679       6  0.771203      accord
811       8  0.731899     adapter
388       7  0.831181  affordable
40        1  0.983881        also
...     ...       ...         ...
488       8  0.629794       worst
814       8  0.893313   worstread
167       2  0.932252       worth
1520      2  0.897123   worthread
206       7  0.844619        year

[277 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 10, 5, 2, 8, 3, 9, 6, 4])

In [19]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

mallet_path = "/content/mallet-2.0.8/bin/mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

--2023-04-03 03:26:28--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2023-04-03 03:26:28--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  16.7MB/s    in 0.9s    

2023-04-03 03:26:29 (16.7 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.

AttributeError: ignored

In [22]:
import gensim
import os
import urllib.request
import zipfile
mallet_path = "/content/mallet-2.0.8/bin/mallet"
if not os.path.exists(mallet_path):
    # Download Mallet
    url = "http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip"
    filename = "mallet.zip"
    urllib.request.urlretrieve(url, filename)

    # Extract Mallet
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)


AttributeError: ignored

In [25]:
!pip uninstall gensim
#!pip install gensim







Found existing installation: gensim 4.3.1
Uninstalling gensim-4.3.1:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/gensim-4.3.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/gensim/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/commands/uninstall.py", line 97, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/req/req_install.py", line 638, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/req/req_uninstall.py", line 363, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/req/req_uninstall.py", line 403, in _allowe

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here

# Write your code here
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt




In [ ]:

def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts
preprocess_data(data)

In [ ]:
def prepare_corpus(doc_clean):
    """
    Input  : clean document
    Purpose: create term dictionary of our courpus and Converting list of documents (corpus) into Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

In [ ]:
number_of_topics=7
words=10

In [ ]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel

In [ ]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
def plot_graph(doc_clean,start, stop, step):
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    model_list, coherence_values = compute_coherence_values(dictionary, doc_term_matrix,doc_clean,
                                                            stop, start, step)
    # Show graph
    x = range(start, stop, step)
    plt.plot(x, coherence_values)
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence score")
    plt.legend(("coherence_values"), loc='best')
    plt.show()

In [ ]:
start,stop,step=2,12,1
plot_graph(preprocess_data(data),start,stop,step)

In [ ]:
# Write your code here
model=create_gensim_lsa_model(preprocess_data(data),number_of_topics,words)

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
# Write your code here
import pyLDAvis
pyLDAvis.enable_notebook()
!pip install lda2vec





In [ ]:
from google.colab import files
uploades = files.upload()

In [ ]:
npz = np.load(open('/content/topics.pyldavis.npz', 'rb'), allow_pickle=True)
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()

In [ ]:
top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

In [ ]:
import warnings
warnings.filterwarnings('ignore')
prepared_data = pyLDAvis.prepare(dat['topic_term_dists'], dat['doc_topic_dists'], 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')
pyLDAvis.display(prepared_data)

In [ ]:
from sklearn.datasets import fetch_20newsgroups
remove=('headers', 'footers', 'quotes')
texts = fetch_20newsgroups(subset='train', remove=remove).data
data=pd.DataFrame(texts)
data=data.rename(columns={0:'News_text'})
data['News_text']=data['News_text'].str.lower()
data['News_text'].head()

In [ ]:
#Remving the numbers from the news text
for letter in '1234567890.(/':
    data['News_text']= data['News_text'].str.replace(letter,'')
#Remving the single letters (like, "u", "w" etc) from the text
data['News_text']=data['News_text'].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')
#Getting Tokenization of News
tokenizer = RegexpTokenizer(r'\w+')
data['News_text'] = data['News_text'].apply(tokenizer.tokenize)
stopwords_list = stopwords.words('english')
data['News_text']=data['News_text'].apply(lambda x: [item for item in x if item not in stopwords_list])
data['News_text'].head()

In [ ]:
#Description of news text information
all_words = [word for tokens in data['News_text'] for word in tokens]
sentence_lengths = [len(tokens) for tokens in data['News_text']]

VOCAB = sorted(list(set(all_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

In [ ]:
from collections import Counter
counter = Counter(all_words)
counter.most_common(20)

In [ ]:
counted_words = Counter(all_words)

words = []
counts = []
for letter, count in counted_words.most_common(25):
    words.append(letter)
    counts.append(count)
def LDA_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 10);
        word_dict['Words of Topic ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)
data_text=pd.DataFrame(data['News_text'])
data_text=data_text.rename(columns={'News_text':'News_Text'})
train_ = [value[0] for value in data_text.iloc[0:].values]

In [ ]:
num_topics = 20
id2word1 = gensim.corpora.Dictionary(train_)
corpus1 = [id2word1.doc2bow(text) for text in train_]
#Calling the LDA model and feeding the calculated text, dictionary and topics


In [ ]:
from gensim.models import ldamodel
lda2vec = ldamodel.LdaModel(corpus=corpus1, id2word=id2word1, num_topics=num_topics)

In [ ]:
#Calling LDA_topics function and using lda2vec with selective topics
LDA_topics(lda2vec, num_topics)

In [ ]:
#Computing the Topic Coherence with 20
coherence_using_lda2vec = CoherenceModel(model=lda2vec, texts=train_, dictionary=id2word1, coherence='c_v')
coherence = coherence_using_lda2vec.get_coherence()
print('Coherence Score: ', round(coherence, 3))
Coherence Score:  0.595

import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda2vec, 
                              corpus=corpus1, 
                              dictionary=id2word1)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [26]:
# Write your code here
%%capture
!pip install bertopic




In [ ]:
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)
freq = topic_model.get_topic_info(); freq.head(5)
topic_model.get_topic(0)

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

In [ ]:
topic_model.visualize_topics()
topic_model.visualize_distribution(probs[200], min_probability=0.015)
topic_model.update_topics(docs, n_gram_range=(1, 2))
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics
topic_model.get_topic(71)

In [ ]:
# Save model
topic_model.save("my_model")	
# Load model
my_model = BERTopic.load("my_model")
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

query_embedding = model.encode('How big is London')
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [ ]:
# Write your answer here (no code needed for this question)
# Write your answer here (no code needed for this question)
'''The evaluation of Topic Modelling can be assessed based on the concept of Topic Coherence. 
There are several techniques available to obtain topic models, such as Latent Semantic Analysis (LSA), 
Latent Dirichlet Allocation (LDA), lda2vec, and BERTopic.

In LDA, the model requires a dictionary and a corpus as input, with a specified number of topics.
To estimate the number of topics, we can use the LDA visualization tool pyLDAvis and compare coherence scores for different numbers of topics. 
The best number of topics is the one that gives the highest coherence score. 
The plot of coherence score and the number of topics typically shows a decreasing trend in coherence score with an increase in the number of topics. 
The optimal number of topics is usually observed at the point where coherence score growth begins to level off,
 and this number is generally interpretable.

For LSA, it is usually used for dimension reduction or noise reduction techniques.
To determine the number of topics, we can also use the Topic coherence measure, where a high value indicates a good topic model. 
The plot of coherence score and the number of topics using LSA algorithm typically shows an optimal coherence score at a specific number of topics.
From the analysis, it can be concluded that LDA gives better results than LSA.

.'''


